In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !pip install -q -U bitsandbytes # 모델을 경량화해서 로드
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git # LoRA 튜닝을 하게 해주는 라이브러리
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q datasets
# !pip install trl==0.4.7
# !pip install accelerate
# !pip install sentence-transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 15.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (

In [1]:
from transformers import AutoTokenizer, LlamaForCausalLM, StoppingCriteria, StoppingCriteriaList
import torch
import argparse
import os
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import warnings
import re
warnings.filterwarnings("ignore")

In [2]:
test_df = pd.read_csv('../test.csv')
# test_df = pd.read_csv('/content/drive/MyDrive/데이콘/test.csv')
def remove_quotes(text):
    text = re.sub(r'"', '', text)
    return text
test_df['질문'] = test_df['질문'].apply(remove_quotes)
test_df

,id,질문
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?"
3,TEST_003,"철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있..."
4,TEST_004,도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?
...,...,...
125,TEST_125,분말 소화기를 사용할 때 주의해야 할 사항은 무엇인가요? 그리고 아파트 도배 평수를...
126,TEST_126,압출법 보온판의 가장 큰 장점은 무엇인가요?
127,TEST_127,평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어...
128,TEST_128,석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 ...


In [3]:
adapter_name = 'llama2_lora_train' # Adapter의 폴더 명을 가져와야 함

bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)
# 기본 base model을 load
model = AutoModelForCausalLM.from_pretrained('beomi/llama-2-ko-7b', quantization_config=bnb_config, device_map={'': 0})
# model.config.use_cache = False
model.config.use_cache = True
model.config.pretraining_tp = 1
# tokenizer를 model에 맞게 가져옴
tokenizer = AutoTokenizer.from_pretrained('beomi/llama-2-ko-7b', trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = prepare_model_for_kbit_training(model)

# model에 아까 학습한 adapter를 연결시킴
model = PeftModel.from_pretrained(model, adapter_name)

bin C:\Users\user\anaconda3\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [4]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(46336, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_feature

In [5]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(46336, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_feature

In [6]:
class LocalStoppingCriteria(StoppingCriteria):

    def __init__(self, tokenizer, stop_words=[]):
        super().__init__()

        stops = [tokenizer(stop_word, return_tensors='pt', add_special_tokens=False)['input_ids'].squeeze() for
                 stop_word in stop_words]
        print('stop_words', stop_words)
        print('stop_words_ids', stops)
        self.stop_words = stop_words
        self.stops = [stop.cuda() for stop in stops]
        self.tokenizer = tokenizer

    def _compare_token(self, input_ids):
        for stop in self.stops:
            if len(stop.size()) != 1:
                continue
            stop_len = len(stop)
            if torch.all((stop == input_ids[0][-stop_len:])).item():
                return True

        return False

    def _compare_decode(self, input_ids):
        input_str = self.tokenizer.decode(input_ids[0])
        for stop_word in self.stop_words:
            if input_str.endswith(stop_word):
                return True
        return False

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        input_str = self.tokenizer.decode(input_ids[0])
        for stop_word in self.stop_words:
            if input_str.endswith(stop_word):
                return True
        return False

In [7]:
stop_words = ["<|endoftext|>", "###", "</s>"]
stopping_criteria = StoppingCriteriaList([LocalStoppingCriteria(tokenizer=tokenizer, stop_words=stop_words)])

stop_words ['<|endoftext|>', '###', '</s>']
stop_words_ids [tensor(46332), tensor(835), tensor(2)]


In [8]:
pipe = pipeline(task="text-generation", # 지정된 task에 맞게 model이 돌아감
                model=model, # 아까 학습시킨 adapter에 모델에 연결된 상태
                tokenizer=tokenizer,
                max_length = 600,
                do_sample=True,
                early_stopping=True,
                temperature=0.01,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.eos_token_id,
                top_k=1,
                top_p=0.99,
                repetition_penalty = 1.3,
                stopping_criteria=stopping_criteria,
                framework='pt'
)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

In [11]:
def ask(x, is_input_full=False):
    ans = pipe(
        f"<s>### Instruction: \n{x} \n\n### Response: "
    )
    answer = ans[0]['generated_text'].split('### Response: ')[1].strip()
    del ans
    torch.cuda.empty_cache()
    return answer

In [11]:
test_df.loc[0, "질문"]

'방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?'

In [12]:
answer = ask(test_df.loc[0, "질문"])
print(answer)

방청페인트는 다양한 종류가 있습니다. 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트 등이 있으며 각각의 특징에 따라 선택할 수 있습니다. 이러한 페인트들은 특정한 용도에 맞게 사용되며, 건축물의 외부 환경이나 요구사항에 부합하는 특성을 가지고 있습니다. 예를 들어 햇빛이 강한 곳에서는 알루민트 페인트나 역청질페인트와 같은 제품이 적합하며, 습기가 많은 장소에는 산화아연과 같이 내구성이 뛰어난 페인트가 효과적일 것입니다. 따라서 상황에 맞는 적절한 페인트 종류를 선정하여 사용할 필요가 있습니다.  이러한 상황에서 방청페인트 중 알미늄페인트가 가장 적합합니다. 이 제품은 부식로부터 보호해주는 특수 성분인 알미늄 가루를 포함하고 있어 강하면서도 내구성과 내부 마감을 증진시키는 장점을 갖고 있습니다. 이에 반해 광명단페인트 등은 빛깔 있는 외관을 제공해주며, 방청산화철페인트는 철근 구조물에 사용하기 좋은 제품입니다. 추가로, 역청질페인트 역시 특정 재료 또는 구조물 등에 적합하게 적용됩니다. 그러므로 필요한 상황과 목적에 맞추어 올바른 페인트 종류를 선택해야 합니다.   만약 건물 주변환경이 특히 노출된 부분이 많거나 오랜 기간 동안 노출될 우려가 있다면, 방청페인트 도장 작업을 고려해볼 만 할 것 입니다. 이는 페인트 제품들의 내구성 및 성능에도 영향을 미치기 때문입니다. 때문에 전문가와의 상담 후 결정하시는 것을 권유 드립니다.  알미늄페인트의 경우 주로 철강재 등 금속재의 표면처리에 많이 쓰이며, 탄산가스 등의 침투로 인한 부식을 방지하기 위해 개발되었습니다. 그래서 실내외 벽체나 지붕 등 금속구조물의 보강용으로 많이 활용되고 있고, 이외에도 목조주택에서도 사용되는 경우가 많습니다. 덧붙여, 방청효과 측면에서 매우 뛰어나기 때문에 외부에서의 공격을 방어하는데도 유용하다고 알려져있습니다.  따라서 방청페인트 중에서 알미늄페인트를 추천드리는 바입니다. 해당 제품은 다양한 색상을 보유하고 있어서 디자인적인 면에서도 활용할 수 있겠습니다. 다만, 알미톤페인

### model inference

In [12]:
preds = []
for t in tqdm(test_df['질문'], total = len(test_df)):
    result = ask(t)
    find = result.find('감사합니다.')
    if find != -1:
        result = result[:find+6]
    find = result.find('#')
    if find != -1:
        result = result[:find]
    preds.append(result.strip())

  0%|          | 0/130 [00:00<?, ?it/s]

In [13]:
preds[:5]

['방청페인트는 다양한 종류가 있습니다. 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트 등이 있으며 각각의 특징에 따라 선택할 수 있습니다. 이러한 페인트들은 특정한 용도에 맞게 사용되며, 건축물의 외부 환경이나 요구사항에 부합하는 특성을 가지고 있습니다. 예를 들어 햇빛이 강한 곳에서는 알루민트 페인트나 역청질페인트와 같은 제품이 적합하며, 습기가 많은 장소에는 산화아연과 같이 내구성이 뛰어난 페인트가 효과적일 것입니다. 따라서 상황에 맞는 적절한 페인트 종류를 선정하여 사용할 필요가 있습니다.  이러한 상황에서 방청페인트 중 알미늄페인트가 가장 적합합니다. 이 제품은 부식로부터 보호해주는 특수 성분인 알미늄 가루를 포함하고 있어 강하면서도 내구성과 내부 마감을 증진시키는 장점을 갖고 있습니다. 이에 반해 광명단페인트 등은 빛깔 있는 외관을 제공해주며, 방청산화철페인트는 철근 구조물에 사용하기 좋은 제품입니다. 추가로, 역청질페인트 역시 특정 재료 또는 구조물 등에 적합하게 적용됩니다. 그러므로 필요한 상황과 목적에 맞추어 올바른 페인트 종류를 선택해야 합니다.   만약 건물 주변환경이 특히 노출된 부분이 많거나 오랜 기간 동안 노출될 우려가 있다면, 방청페인트 도장 작업을 고려해볼 만 할 것 입니다. 이는 페인트 제품들의 내구성 및 성능에도 영향을 미치기 때문입니다. 때문에 전문가와의 상담 후 결정하시는 것을 권유 드립니다.  알미늄페인트의 경우 주로 철강재 등 금속재의 표면처리에 많이 쓰이며, 탄산가스 등의 침투로 인한 부식을 방지하기 위해 개발되었습니다. 그래서 실내외 벽체나 지붕 등 금속구조물의 보강용으로 많이 활용되고 있고, 이외에도 목조주택에서도 사용되는 경우가 많습니다. 덧붙여, 방청효과 측면에서 매우 뛰어나기 때문에 외부에서의 공격을 방어하는데도 유용하다고 알려져있습니다.  따라서 방청페인트 중에서 알미늄페인트를 추천드리는 바입니다. 해당 제품은 다양한 색상을 보유하고 있어서 디자인적인 면에서도 활용할 수 있겠습니다. 다만, 알미톤

In [14]:
preds[-5:]

['분말 소화기의 장점은 비교적 저렴한 가격과 간단한 사용 방법이며, 압력계와 불연소재로 구성된 안전기구이기 때문에 안심하고 사용할 수 있습니다. 그러나 단점으로는 분말을 뿌리기 위해 강한압력이 필요하기 때문에 한 번에 많은 양을 뿌리기가 어렵고, 압력에 의해 반동이 발생할 경우 다칠 위험이 있다는 점입니다. 또한 분말이 완전히 소진되지 않은 채로 보관하거나 하면 굳어버릴 가능성이 있으니 주의가 요구됩니다. 따라서 분말 소화기는 안전을 유지하는 데 유용하게 활용될 수 있지만 적절한 관리가 필요하다는 점을 염두해 두시기 바랍니다.  특히, 분말 소화기에는 흔히 무시되는 몇 가지 중요한 관리 요인들이 있는데요 - 그 중에서도 특히 다음의 내용들을 유의하셔야 합니다.   1. 초소형 화재 용품임에도 불구하고 제대로 된 관리를 하지 않는다면 고장의 원인이 될 수 있다.   2. 분말 소화기를 사용하는 동안에는 직사광선이나 고온 환경에서 멀리하여야 한다.   3. 분말 소화기를 사용한 후에는 내부의 약제를 제거하지 않고 보관할 때는 안전하게 캡이 씌워져야 하며, 분말이 완전하게 배출된 후에 다시 사용해햐 한다.   4. 분말 소화기를 이동 시에는 반드시 안전한 방법으로 운반한다.   5. 분말 소화기에 대한 정기적인 점검 및 청소가 필요하다.   6. 분말 소화기의 압력과 온도를 주기적으로 확인하며, 관련 부속품의 교체나 정비가 필요할 땐 전문가의 도움을 받아야 한다.   7. 분말 소화기를 사용하기 전엔 꼭 환기와 질식사고를 예방하기 위한 조치를 취해야한다.   8. 분말 소화기를 보관함에 넣을때는 흔들리지 않도록 해야하며 꽉 끼지 않게 넣어줘야 한다.   9. 분말 소화기를 사용할땐 주변사람들에게 알리고 사용시 주의사항 숙지 해야한다.   10. 분말 소화기를 사용하고 난 뒤엔 충분히 눌러주어야 한다.   이러한 관리들이 적절히 이루어지지 않으면 분말 소화기의 성능을 유지하고 안전을 보장하는데 문제가 있을 것입니다. 올바른 관리 방법을 통해 분말 소화기를 효율적이고

In [19]:
pd.DataFrame({'질문':test_df['질문'],'답변':preds}).to_csv('llama2_lora_train(epoch5)_text.csv', index = False)

### submission

In [16]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = model.encode(preds)
# pred_embeddings = model.encode(seperate_preds)
pred_embeddings.shape

(130, 512)

In [17]:
submit = pd.read_csv('../sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.head()

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.028195,-0.010630,-0.035204,-0.004698,0.078816,0.045277,-0.003816,0.033808,0.000855,...,0.002156,-0.034321,-0.003993,-0.026567,-0.022242,0.002200,0.060209,0.051718,-0.009402,0.036449
1,TEST_001,-0.015267,0.031962,0.002600,-0.001115,0.097380,-0.003280,-0.000254,-0.024340,-0.004540,...,0.008678,-0.014734,0.022658,-0.022281,-0.042278,0.023978,-0.001812,-0.059364,-0.000064,0.049090
2,TEST_002,0.015316,-0.027548,-0.046749,0.006143,0.127866,-0.053530,-0.016439,-0.046631,0.045050,...,-0.016038,-0.032279,0.041694,-0.016836,-0.030388,0.020534,-0.028684,-0.028945,-0.043558,0.100803
3,TEST_003,0.003933,0.027793,-0.037549,0.024963,0.076725,-0.076401,-0.044969,-0.064264,-0.006688,...,-0.024064,-0.040201,0.051388,-0.034828,-0.013229,0.009073,-0.015826,-0.043631,-0.016827,0.019298
4,TEST_004,0.027416,-0.012322,-0.006479,-0.000736,0.097112,-0.005138,0.039447,0.029235,-0.027818,...,0.021272,0.011633,0.026010,0.005951,-0.040456,-0.004368,-0.005130,0.035413,-0.007024,0.047944


In [18]:
submit.to_csv('llama2_lora_train(epoch5).csv', index = False)
pd.read_csv('llama2_lora_train(epoch5).csv')

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.028195,-0.010630,-0.035204,-0.004698,0.078816,0.045277,-0.003816,0.033808,0.000855,...,0.002156,-0.034321,-0.003993,-0.026567,-0.022242,0.002200,0.060209,0.051718,-0.009402,0.036449
1,TEST_001,-0.015267,0.031962,0.002600,-0.001115,0.097380,-0.003280,-0.000254,-0.024340,-0.004540,...,0.008678,-0.014734,0.022658,-0.022281,-0.042278,0.023978,-0.001812,-0.059364,-0.000064,0.049090
2,TEST_002,0.015316,-0.027548,-0.046749,0.006143,0.127866,-0.053530,-0.016439,-0.046631,0.045050,...,-0.016038,-0.032279,0.041694,-0.016836,-0.030388,0.020534,-0.028684,-0.028945,-0.043558,0.100803
3,TEST_003,0.003933,0.027793,-0.037549,0.024963,0.076725,-0.076401,-0.044969,-0.064264,-0.006688,...,-0.024064,-0.040201,0.051388,-0.034828,-0.013229,0.009073,-0.015826,-0.043631,-0.016827,0.019298
4,TEST_004,0.027416,-0.012322,-0.006479,-0.000736,0.097112,-0.005138,0.039447,0.029235,-0.027818,...,0.021272,0.011633,0.026010,0.005951,-0.040456,-0.004368,-0.005130,0.035413,-0.007024,0.047944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,TEST_125,0.006793,-0.077726,0.009070,-0.007896,0.108282,-0.001226,0.055052,-0.013643,0.018230,...,-0.029109,0.019343,0.031434,-0.025546,0.053640,0.009263,0.028441,-0.046160,-0.006637,0.040872
126,TEST_126,-0.019836,-0.042854,-0.055216,0.002099,0.114133,-0.017669,0.019676,0.031714,-0.012748,...,-0.010532,-0.070005,0.056857,-0.041490,0.016432,0.054713,-0.006125,-0.071770,-0.005414,0.094057
127,TEST_127,-0.050065,-0.008485,0.031355,-0.005816,0.089623,-0.023374,0.012901,0.002748,-0.007914,...,-0.041762,-0.061789,0.003831,-0.020114,-0.015658,0.034375,0.000910,0.046129,-0.032812,0.029996
128,TEST_128,0.004712,-0.039422,-0.046734,0.018193,0.103467,-0.072224,0.006717,0.006719,-0.025522,...,-0.023518,-0.011211,0.046586,-0.030761,-0.019604,0.055346,-0.027627,-0.021050,0.030847,0.054058
